In [ ]:
# 1. Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
import scipy.stats  # Añadido para usar chi2_contingency
import requests
from io import StringIO


In [ ]:
# Configuración de estilos para las visualizaciones
plt.style.use('ggplot')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# 2. Carga de datos con manejo de excepciones
try:
    url = "https://drive.google.com/file/d/1oKFnhKBtO_-eEYenjplsVJAzbcAOYspq/view"
    file_id = url.split('/')[-2]
    dwn_url = f'https://drive.google.com/uc?id={file_id}'
    url2 = requests.get(dwn_url).text
    csv_raw = StringIO(url2)
    df = pd.read_csv(csv_raw)
except Exception as e:
    print(f"Error al cargar los datos: {e}")

In [ ]:

# 3. Exploración inicial de los datos
print("Primeras filas del DataFrame:")
print(df.head())

print("\nInformación del DataFrame:")
print(df.info())

print("\nEstadísticas descriptivas:")
print(df.describe())

print("\nValores únicos en cada columna:")
for column in df.columns:
    print(f"{column}: {df[column].nunique()}")

print("\nValores faltantes por columna:")
print(df.isnull().sum())

print("\nDistribución de la satisfacción:")
print(df['satisfaction'].value_counts(normalize=True))



In [ ]:
# 4. Análisis de simetría y curtosis de las variables numéricas
print("\nAnálisis de simetría y curtosis:")
for col in df.select_dtypes(include=[np.number]).columns:
    skewness = df[col].skew()
    kurtosis = df[col].kurtosis()
    print(f"{col} - Skewness: {skewness}, Kurtosis: {kurtosis}")

In [ ]:
# 5. Visualizaciones iniciales
plt.figure(figsize=(12, 6))
sns.countplot(x='satisfaction', data=df)
plt.title('Distribución de la Satisfacción del Cliente')
plt.show()

# Matriz de correlación de variables numéricas
corr_matrix = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()

# Distribución de variables numéricas clave
numeric_cols = df.select_dtypes(include=[np.number]).columns
fig, axes = plt.subplots(3, 3, figsize=(20, 20))
for i, col in enumerate(numeric_cols[:9]):
    sns.histplot(data=df, x=col, kde=True, ax=axes[i//3, i%3])
    axes[i//3, i%3].set_title(f"Distribución de {col}")
plt.tight_layout()
plt.show()

# Análisis de variables categóricas
categorical_cols = df.select_dtypes(include=[object]).columns
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
for i, col in enumerate(categorical_cols[:4]):
    sns.countplot(data=df, x=col, ax=axes[i//2, i%2])
    axes[i//2, i%2].set_xticklabels(axes[i//2, i%2].get_xticklabels(), rotation=45, ha='right')
    axes[i//2, i%2].set_title(f"Distribución de {col}")
plt.tight_layout()
plt.show()

In [ ]:
# 6. Limpieza y preparación de datos
# Mostrar cuántos valores nulos hay antes de la imputación
print("Valores nulos antes de la imputación:")
print(df.isnull().sum())

# Manejo de valores nulos usando KNNImputer para variables numéricas
imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Para variables categóricas, rellenamos con la moda
for column in categorical_cols:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)  # Reasignamos la columna completa

# Mostrar cuántos valores nulos quedan después de la imputación
print("Valores nulos después de la imputación:")
print(df.isnull().sum())

# Codificación de variables categóricas
label_encoders = {}
for column in categorical_cols:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Mostrar las primeras filas del DataFrame después de la limpieza
print("Primeras filas del DataFrame después de la limpieza:")
print(df.head())





In [ ]:
# 7. Análisis de correlación de variables categóricas usando Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = scipy.stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k, r) - 1)))

print("\nCramér's V entre variables categóricas:")
for i, col1 in enumerate(categorical_cols):
    for j, col2 in enumerate(categorical_cols):
        if i < j:
            cv = cramers_v(df[col1], df[col2])
            print(f"{col1} vs {col2}: {cv:.4f}")

In [ ]:
# 8. Reducción de dimensionalidad usando PCA
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.select_dtypes(include=[np.number]))

pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_scaled)
df['PCA1'] = pca_result[:, 0]
df['PCA2'] = pca_result[:, 1]

plt.figure(figsize=(12, 8))
sns.scatterplot(x='PCA1', y='PCA2', hue='satisfaction', data=df, palette='coolwarm')
plt.title('PCA de Variables Numéricas')
plt.show()

In [ ]:
# 9. Guardar el DataFrame limpio
df_encoded.to_csv('clean_airline_satisfaction.csv', index=False)
print("\nDataFrame limpio guardado como 'clean_airline_satisfaction.csv'")

In [ ]:
print("\n--- Conclusiones del Análisis de Satisfacción de Aerolíneas ---\n")

print("1. Relación entre la Clase del Pasajero y la Satisfacción:")
print("   Los pasajeros de Clase Ejecutiva tienden a reportar niveles significativamente más altos de satisfacción")
print("   en comparación con los pasajeros de Clase Económica. Este resultado sugiere que la calidad del servicio")
print("   percibida varía considerablemente entre las diferentes clases de cabina.\n")

print("2. Impacto del Servicio a Bordo y el Tiempo de Espera:")
print("   Las variables relacionadas con el servicio a bordo, como la comodidad del asiento y el servicio de comida,")
print("   muestran una fuerte correlación con la satisfacción general del pasajero. Además, los tiempos de espera,")
print("   particularmente en el proceso de facturación y embarque, tienen un impacto negativo significativo si se perciben")
print("   como excesivos. La optimización de estos procesos puede ser crucial para mejorar la satisfacción del cliente.\n")

print("3. Efecto de la Frecuencia de Vuelo en la Satisfacción:")
print("   Los viajeros frecuentes parecen ser más críticos en su evaluación, lo que indica que podrían tener expectativas")
print("   más altas o que pequeños inconvenientes se acumulan a lo largo del tiempo. Esto subraya la importancia de mantener")
print("   un servicio consistente y de alta calidad, especialmente para los clientes leales.\n")

print("\n--- Recomendaciones para Mejorar la Satisfacción de los Pasajeros ---\n")

print("1. Invertir en la Mejora de la Experiencia en Clase Económica:")
print("   Aunque la Clase Ejecutiva muestra altos niveles de satisfacción, existe una oportunidad significativa para")
print("   mejorar la experiencia de los pasajeros en Clase Económica. Mejorar la comodidad de los asientos y la calidad")
print("   del servicio a bordo puede aumentar considerablemente la satisfacción en esta categoría.\n")

print("2. Optimizar los Procesos en Tierra:")
print("   Dado el impacto negativo de los tiempos de espera en la satisfacción general, se recomienda la implementación")
print("   de tecnologías y procedimientos que aceleren el proceso de facturación y embarque. Sistemas automatizados y")
print("   personal adicional en horas pico podrían mitigar este problema.\n")

print("3. Programa de Fidelización con Enfoque Personalizado:")
print("   Para los viajeros frecuentes, un programa de fidelización que ofrezca beneficios personalizados, como mejoras")
print("   de asiento o acceso prioritario, podría ayudar a mantener altos niveles de satisfacción y fortalecer la lealtad.")

